In [1]:
# import dependencies
import pandas as pd
import numpy as np
import tableauhyperapi as tab
import os
import win32com.client
from shutil import rmtree, move
from dateutil.relativedelta import relativedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from zipfile import ZipFile
from pathlib import Path
from time import sleep
import pyodbc
from tqdm import tqdm
import sqlalchemy


In [2]:
tableau_url = '' # Network URL

In [ ]:
cwd = os.getcwd()

data_folder = os.path.join(cwd, 'Data')
dashboard_file = 'Behavior Analyzer Agent Performance IV VR.twbx'
dasboard_path = os.path.join(data_folder, dashboard_file)

hyper_folder = os.path.join(data_folder, 'hyper')
if os.path.exists(hyper_folder) == False:
    os.makedirs(hyper_folder)
else:
    rmtree(hyper_folder)
    os.makedirs(hyper_folder)


download_folder = os.path.join(Path.home(), 'Downloads')
download_path = os.path.join(download_folder, dashboard_file)

while os.path.exists(download_path) == False:
    print(f'Dashboard has not been detected.')
    print(f'Please download the report from: {tableau_url}')
    print('-'*25)
    input('Press Enter when completed')
else:
    print('Dashboard detected. Continuing with the script.')
    move(download_path, dasboard_path)


In [4]:
with ZipFile(dasboard_path, 'r') as zip_ref:
    zip_ref.extractall(hyper_folder)
    print("Tableau File Extracted")


Tableau File Extracted


In [ ]:
database_folder = os.path.join(hyper_folder, 'Data', 'Extracts')
database_folder_contents = os.listdir(database_folder)
hyper_files = [value for value in database_folder_contents if value.endswith('.hyper')]

if len(hyper_files) > 1:
    print('there is more than one hyper file')

database_path = os.path.join(database_folder, hyper_files[0])
print(database_path)

In [6]:
with tab.HyperProcess(telemetry=tab.Telemetry.DO_NOT_SEND_USAGE_DATA_TO_TABLEAU) as hyper:
    with tab.Connection(hyper.endpoint, database=database_path) as connection:
        table_name = tab.TableName("Extract", "Extract")
        table_definition = connection.catalog.get_table_definition(name=table_name)
        column_names = [str(value.name)[1:-1] for value in table_definition.columns]
        results = connection.execute_list_query(query=f"SELECT agent_id, behavior, caldt, deno_mediafilecount, num_mediafilecount FROM {table_name} WHERE lob = 'Video Repair'")

In [7]:
column_names = ['Agent Id','Behavior','Caldt','Denominator Count','Numerator Count']
df = pd.DataFrame(data=results, columns=column_names)
df = df.dropna(how='all', subset=['Denominator Count','Numerator Count']).reset_index(drop=True)
df['Caldt'] = df['Caldt'].astype(str)
df['Caldt'] = pd.to_datetime(df['Caldt']).dt.date
row_count = df.shape[0]

print(f'Total rows extracted: {row_count:,}')

Total rows extracted: 651,273


In [8]:
earliest_date = df['Caldt'].min()
earliest_date_str = earliest_date.strftime("%m/%d/%Y")

clean_table_query = f'''
DELETE FROM GVPOperations.VID.OAI_VR_Export where Caldt >= '{earliest_date.strftime("%m/%d/%Y")}'
'''

In [9]:
print(f'Earliest date from Tableau: {earliest_date.strftime("%m/%d/%Y")}')

2023-03-07


In [10]:
server_name = '' # Network Server Name
db_name = 'GVPOperations'

engine = sqlalchemy.create_engine(f'mssql+pyodbc://@{server_name}/{db_name}?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server', fast_executemany=True)
conn = engine.connect()
print('Database connected')

In [ ]:
from sqlalchemy import event
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
       conn, cursor, statement, params, context, executemany
        ):
            if executemany:
                cursor.fast_executemany = True

In [11]:
conn.execute(sqlalchemy.sql.text(clean_table_query))
conn.commit()

print('Old Rows Deleted')

Old Rows Deleted


In [13]:
print(f'Writing {row_count:,} to database')
df.to_sql(name='OAI_VR_Export', con=engine, schema='VID', if_exists='append', index=False)

147

In [15]:
conn.close()
engine.dispose()